In [30]:
import numpy as np

In [31]:
def Relu(x):
    return (np.abs(x)+x)/2


In [32]:
Relu(np.array([-3,0,2]))

array([0., 0., 2.])

In [ ]:
class Layer:

    def Relu(x):
        return (np.abs(x)+x)/2

    def __init__(self,dim_in,dim_out,init_fn,act_fn=Relu):
        self.weights=init_fn(dim_in,dim_out)
        self.bias=init_fn(1,dim_out)
        self.act_fn=act_fn


    def fw(self,input):
        output=self.weights@input + self.bias
        if self.act_fn:
            output=self.act_fn(output)
        return output    




In [ ]:
class FFNN:

    def __init__(self,input_size,hidden_sizes,output_size, init_fn,act_fn):
        self.layers=[]
        self.layers.append(Layer(dim_in=input_size,dim_out=hidden_sizes[0],init_fn=init_fn,act_fn=act_fn))
        for i in range(1,len(hidden_sizes)):
            self.layers.append(Layer(dim_in=hidden_sizes[i-1],dim_out=hidden_sizes[i],init_fn=init_fn,act_fn=act_fn))
        self.layers.append(Layer(dim_in=hidden_sizes[-1],dim_out=output_size,init_fn=init_fn,act_fn=False))

    

    def forward(self,input):
        output=input
        for layer in self.layers:
            output=layer.fw(output)
        return output






In [47]:
test=FFNN(input_size=3,hidden_sizes=[2,2,4,4],output_size=3,act_fn=Relu)

In [48]:
test_input=np.array([3,14,5])


In [49]:
print(test.forward(test_input))

[-2.68200976  1.6103503  -1.73161803]
